<a href="https://colab.research.google.com/github/itsTurner/ben-learns-ml/blob/main/Password_AutoEncoder_Decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Necessary import statements

In [100]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import math
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA as skPCA

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import optimizers
from tensorflow.keras import losses

Import frequent words data

In [146]:
fullWordsList = pd.read_csv('unigram_freq.csv')
wordsList = fullWordsList[['word']]
wordsArray = np.array(wordsList['word'])


# for index, element in enumerate(wordsArray):
#   try:
#     if len(element) == 6:
#       #print("converter",converter(element))
#       fiveLengthValues[index] = converter(element)
#       #print("five length",fiveLengthValues[index])
#       #fiveLengthArray = np.append(fiveLengthArray, element)
#       # print(converter(list(element)))
#   except:
#     print(element, "fucked it up")
# #print(fiveLengthValues)


#print()
fiveLengthArray = formatter(wordsArray, 6)
fiveLengthValues = np.zeros((len(fiveLengthArray), 6))

fiveLengthValues = converter(fiveLengthArray, 6)
print(fiveLengthValues)
#print(fiveLengthArray)


nan fucked it up with error of object of type 'float' has no len()
nan fucked it up with error of object of type 'float' has no len()
[[0.69230769 0.15384615 0.         0.65384615 0.07692308 0.26923077]
 [0.53846154 0.5        0.42307692 0.30769231 0.5        0.15384615]
 [0.57692308 0.15384615 0.53846154 0.57692308 0.42307692 0.15384615]
 ...
 [0.23076923 0.53846154 0.53846154 0.19230769 0.15384615 0.42307692]
 [0.23076923 0.53846154 0.53846154 0.11538462 0.11538462 0.23076923]
 [0.23076923 0.53846154 0.42307692 0.42307692 0.23076923 0.53846154]]


In [102]:
print(fiveLengthValues[100])

[0. 0. 0. 0. 0. 0.]


Take input and format to be used generalized

In [103]:
print("Input password of length 5")
password = "hello"
password = password.lower()

passwordValue = np.empty(len(password))

# passwordArray = np.array([])
# for index, element in enumerate(reversed(password)):
#   passwordArray = np.append(element, passwordArray)

alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

for index, element in enumerate(password):
  passwordValue[index] = alphabet.index(element)/len(alphabet)

print(passwordValue)

Input password of length 5
[0.26923077 0.15384615 0.42307692 0.42307692 0.53846154]


Function to take a string and convert to the values

In [144]:
def calculator(input):
  values = np.empty(len(input))
  for index, element in enumerate(input):
    values[index] = alphabet.index(element)/len(alphabet)
  return values


def formatter(input, length):
  formattedArray = np.array([])
  for index, element in enumerate(input):
    try:
      if len(element) == length:
        element = [element]
        x = np.array(element)

        formattedArray = np.append(formattedArray, x, axis=0)
    except Exception as e:
      print(element, "fucked it up", "with error of", e)
      #print("element is shape", np.array(list(element)).shape)
  return formattedArray

def converter(input, length):
  valuesArray = np.zeros((len(input), length))
  for index, element in enumerate(input):
    try:
      valuesArray[index] = calculator(element)
    except Exception as e:
      print(element, "fucked it up", "with error of", e)
  return valuesArray


Generate array of strings of length 5

In [105]:
def generator(length):
  string = []
  for i in range(length):
    string.append(random.choice(alphabet))
  return string
print(generator(5))

['p', 'k', 'w', 'h', 'g']


Next initialize your own data set of random strings that have been converted

In [106]:
# x_train = np.array(np.zeros(([1000, 6])))
# #y_train = np.array(np.zeros(([1000, 1])))
# for i in range(len(x_train)):
#   values = converter(fiveLengthArray[i])
#   values = np.array([values])
#   #expectedPrediction = encoder.predict(values)
#   x_train[i] = values
#   #y_train[i] = expectedPrediction

# print(x_train)

##Tavin Autoencoder Implementation

In [147]:
encoder_input_dim = (6,)
latent_space_dim = 1

encoder_input = layers.Input(shape=encoder_input_dim, name='encoder_input')

x = encoder_input

encoder_layer_size = [4, 2]

for i in range(len(encoder_layer_size)):
  dense_layer = layers.Dense(encoder_layer_size[i], name="encoder_conv_" + str(i))
  x = dense_layer(x)
  x = layers.LeakyReLU()(x)
  #x = activations.selu(x)

encoder_output = layers.Dense(latent_space_dim)(x)

encoder = keras.Model(encoder_input, encoder_output)

#encoder.summary()

In [148]:
decoder_input = layers.Input(shape=(latent_space_dim,), name='decoder_input')

x = decoder_input

decoder_layer_size = [2, 4, 6]

for i in range(len(decoder_layer_size)):
  dense_layer = layers.Dense(decoder_layer_size[i], name="encoder_conv_" + str(i))
  x = dense_layer(x)
  x = activations.sigmoid(x)

decoder_output = x

decoder = keras.Model(decoder_input, decoder_output)

#decoder.summary()

In [149]:
model_input = encoder_input
model_output = decoder(encoder_output)

model = keras.Model(model_input, model_output)

#model.summary()

In [150]:
learning_rate= 0.4

optimizer = optimizers.Adam(lr=learning_rate)

def r_loss(y_true, y_pred):
  return keras.mean(keras.square(y_true - y_pred))

model.compile(optimizer=optimizer, loss=losses.MeanSquaredError())

In [151]:
model.fit(
    x = fiveLengthValues,
    y = fiveLengthValues,
    # x = x_train,
    # y = x_train,
    batch_size = 10000,
    shuffle = True,
    epochs = 100,
    verbose = 0
)

In [152]:
sampleString = ["r", "a", "n", "d", "o", "m"]
#, "g", "h", "i", "j", "k", "l", "m", "n", "o", "p"]
sampleValues = calculator(sampleString)
sampleValues = np.array([sampleValues])

midway = encoder.predict([sampleValues])
print(midway)
output = decoder.predict([midway])
print(output)

output = output*len(alphabet)
print(output)


[[-8.968789]]
[[0.4069006  0.35988975 0.4427969  0.41301376 0.35681272 0.45948163]]
[[10.579416  9.357134 11.512719 10.738358  9.27713  11.946523]]
